In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Load data
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

# Combine for feature engineering
full_data = [train, test]

# ---- FEATURE ENGINEERING ----

# Extract Title
for dataset in full_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Replace rare titles
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady','Countess','Capt','Col','Don','Dr',
                                                 'Major','Rev','Sir','Jonkheer','Dona'],'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

# Title mapping
title_map = {"Mr":1, "Miss":2, "Mrs":3, "Master":4, "Rare":5}
for dataset in full_data:
    dataset['Title'] = dataset['Title'].map(title_map)
    dataset['Title'] = dataset['Title'].fillna(0)

# Convert Sex
for dataset in full_data:
    dataset['Sex'] = dataset['Sex'].map({"male":0, "female":1})

# ⭐ FIXED: correct way to fill Age using transform
for dataset in full_data:
    dataset['Age'] = dataset['Age'].fillna(
        dataset.groupby("Title")['Age'].transform("median")
    )

# FamilySize + IsAlone
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = (dataset['FamilySize'] == 1).astype(int)

# Embarked
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    dataset['Embarked'] = dataset['Embarked'].map({'S':0,'C':1,'Q':2})

# Fare
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

# Select features
features = ['Pclass','Sex','Age','Fare','FamilySize','IsAlone','Embarked','Title']

X = train[features]
y = train['Survived']
test_features = test[features]

# Model
model = RandomForestClassifier(
    n_estimators=500,
    max_depth=6,
    random_state=42
)

model.fit(X, y)
predictions = model.predict(test_features)

# Submission file
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predictions
})

submission.to_csv("submission.csv", index=False)
submission.head()


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
